# Scientia-Model


## Import Libraries and Configuration


In [ ]:
from marshal import dumps, loads
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException
from mpstemmer import MPStemmer
from numpy import argsort
from polars import col, read_csv, Series, String
from re import sub
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from types import FunctionType

file_path_dataset = "temp/repository_pnj_20212023.csv"
file_path_select_feature = "temp/01_select_feature.csv"
file_path_lowercase = "temp/02_lowercase.csv"
file_path_remove_empty_abstract = "temp/03_remove_empty_abstract.csv"
file_path_remove_dash_abstract = "temp/04_remove_dash_abstract.csv"
file_path_remove_same_title_abstract = "temp/05_remove_same_title_abstract.csv"
file_path_merge = "temp/06_merge.csv"
file_path_remove_non_alphabet = "temp/07_remove_non_alphabet.csv"
file_path_unique = "temp/08_unique.csv"
file_path_add_lang = "temp/09_add_lang.csv"
file_path_filter_lang = "temp/10_filter_lang.csv"
file_path_stem = "temp/11_stem.csv"
file_path_remove_stopwords = "temp/12_remove_stopwords.csv"
file_path_dataset_clean = "temp/repository_pnj_20212023clean.csv"
file_path_model = "temp/model.marshall"


def print_file_table(output):
    print(f"./{output}", read_csv(output))

## Preprocessing


### 1. Select Features


In [ ]:
read_csv(file_path_dataset).columns

In [ ]:
input = file_path_dataset
output = file_path_select_feature

read_csv(input).with_columns(
    [col("title").alias("f1"), col("abstract").alias("f2")]
).write_csv(output)

print_file_table(output)

### 2. Lowercase


In [ ]:
input = file_path_select_feature
output = file_path_lowercase

read_csv(input).with_columns(
    [
        col("f1").str.to_lowercase(),
        col("f2").str.to_lowercase(),
    ]
).write_csv(output)

print_file_table(output)

### 3. Remove Missing Values


#### 3.1. Remove Empty Abstract


In [ ]:
input = file_path_lowercase
output = file_path_remove_empty_abstract

read_csv(input).with_columns(col("f2").replace("", None)).drop_nulls().write_csv(output)

print_file_table(output)

#### 3.2. Remove Dash Abstract


In [ ]:
input = file_path_remove_empty_abstract
output = file_path_remove_dash_abstract

read_csv(input).with_columns(col("f2").replace("-", None)).drop_nulls().write_csv(
    output
)

print_file_table(output)

#### 3.3. Remove Same Title Abstract


In [ ]:
input = file_path_remove_dash_abstract
output = file_path_remove_same_title_abstract

read_csv(input).filter(col("f1") != col("f2")).write_csv(output)

print_file_table(output)

### 4. Merge Title Abstract


In [ ]:
input = file_path_remove_same_title_abstract
output = file_path_merge

read_csv(input).with_columns((col("f1") + " " + col("f2")).alias("f")).drop(
    ["f1", "f2"]
).write_csv(output)

print_file_table(output)

### 5. Remove Non Alphabetical Characters


In [ ]:
input = file_path_merge
output = file_path_remove_non_alphabet

read_csv(input).with_columns(col("f").str.replace_all(r"[^a-z]+", " ")).write_csv(
    output
)

print_file_table(output)

### 6. Remove Duplicate


In [ ]:
input = file_path_remove_non_alphabet
output = file_path_unique

read_csv(input).unique("f").write_csv(output)

print_file_table(output)

### 7. Filter Based on Language


#### 7.1. Add Column Language


In [ ]:
input = file_path_unique
output = file_path_add_lang


def detect_language(text):
    try:
        return detect(text)
    except LangDetectException:
        return "unknown"


read_csv(input).with_columns(
    col("f").map_elements(detect_language, return_dtype=String).alias("lang")
).write_csv(output)

print_file_table(output)

#### 7.2. Remove Non-"Bahasa Indonesia"


In [ ]:
input = file_path_add_lang
output = file_path_filter_lang

read_csv(input).filter(col("lang") == "id").drop("lang").write_csv(output)

print_file_table(output)

### 8. Stemming


In [ ]:
input = file_path_filter_lang
output = file_path_stem


def stem_text(text):
    return MPStemmer().stem_kalimat(text)


read_csv(input).with_columns(
    col("f").map_elements(stem_text, return_dtype=String)
).write_csv(output)

print_file_table(output)

### 9. Remove Stopwords


In [ ]:
input = file_path_stem
output = file_path_remove_stopwords


def remove_stopwords(text):
    return StopWordRemoverFactory().create_stop_word_remover().remove(text)


read_csv(input).with_columns(
    col("f").map_elements(remove_stopwords, return_dtype=String)
).write_csv(output)

print_file_table(output)

### 10. Sort


In [ ]:
input = file_path_remove_stopwords
output = file_path_dataset_clean

read_csv(input).sort("url").write_csv(output)

print_file_table(output)

## Modeling


In [ ]:
# Read the dataset from a CSV file
df = read_csv(file_path_dataset_clean)

# Initialize a TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Fit the TF-IDF model to the 'f' column of the dataset
tfidf_matrix = vectorizer.fit_transform(df["f"].to_list())


def find_similar_documents(title: str, abstract: str, top_n: int):
    # Combine title and abstract into one string and convert to lowercase
    combined_text = (title + " " + abstract).lower()

    # Remove all non-alphabetic characters and replace them with spaces
    cleaned_text = sub(r"[^a-z]", " ", combined_text)

    # Split the cleaned text into words and rejoin to remove extra spaces
    trimmed_text = " ".join(cleaned_text.split())

    # Apply stemming to the cleaned text
    stemmed_text = MPStemmer().stem_kalimat(trimmed_text)

    # Remove stop words from the stemmed text
    processed_text = (
        StopWordRemoverFactory().create_stop_word_remover().remove(stemmed_text)
    )

    # Compute cosine similarity between the processed text and the TF-IDF matrix
    cosine_scores = cosine_similarity(
        vectorizer.transform([processed_text]), tfidf_matrix
    ).flatten()

    # Get the indices of the top N most similar documents
    top_n_indices = argsort(cosine_scores)[-top_n:][::-1]

    # Return the top N most similar documents along with their similarity scores
    return (
        df[top_n_indices]
        .with_columns(
            Series(name="similarity", values=cosine_scores[top_n_indices] * 100)
        )
        .drop("f")
    )


with open(file_path_model, "wb") as file:
    file.write(dumps(find_similar_documents.__code__))

print(f"Model saved to ./{file_path_model}")

In [ ]:
with open(file_path_model, "rb") as file:
    model = file.read()

print(
    FunctionType(loads(model), globals())(
        title="APLIKASI VARIABLE SPEED DRIVE ATV610U75N4 PADA KONTROL MOTOR AC 3 FASA BERBASIS PLC",
        abstract="Penggunaan Variable Speed Drive (VSD) pada motor induksi tiga fasa dapat mengurangi konsumsi energi yang dibutuhkan oleh peralatan secara signifikan. Pengaturan kecepatan motor induksi dapat dilakukan dengan cara mengatur tegangan sumber atau frekuensi sumber yang dimaksudkan untuk mendapatkan kecepatan putaran dan torsi motor yang diinginkan atau sesuai dengan kebutuhan. Dengan pengaplikasian Variable Speed Drive (VSD) kecepatan motor dapat dikontrol dan beroperasi dengan mode multi speed. Panel kontrol kecepatan motor ini digunakan untuk memantau dan mengatur kecepatan motor induksi tiga fasa dengan komponen utama yang terdiri dari inverter / VSD tipe ATV610U75N4, PLC, SCADA, dan motor induksi sebagai output. Inverter atau VSD (Variable Speed Drive) digunakan sebagai komponen pengatur kecepatan operasi motor induksi tiga fasa dengan mengatur frekuensi keluaran. PLC sebagai pengontrol urutan dan mengatur input output yang kemudian diproses untuk menghasilkan output yang diinginkan. Sedangkan SCADA digunakan sebagai pengendali jarak jauhnya. Pengaturan kecepatan motor dilakukan dengan mengatur besar frekuensi di inverter. Semakin besar nilai frekuensi maka putaran motor akan lebih cepat. Pada VSD tipe ATV610U75N4 terdapat jenis gangguan fasa loss (Output Phase Loss). Gangguan OPL dari inverter dapat membuat motor tidak dapat bekerja karena daya motor yang tidak memenuhi batas pengaturan (setting) pada inverter yaitu 1,5 kW sedangkan motor yang digunakan sebesar 0,25 kW. Kata Kunci : Frekuensi, Motor, PLC, SCADA, VSD",
        top_n=1000,
    )
)